In [54]:
import pandas as pd
import requests

In [55]:
PREPARED_DATASET_PATH = './shaslyk.csv'

In [56]:
url = 'https://api.delivery-club.ru/api1.2/vendors/search'

In [57]:
params = {'latitude': '55.7577374', 'longitude': '37.6164793', 'query': 'шашлыки',}

In [58]:
response = requests.get(url=url, params=params)
print(response.status_code)

200


In [59]:
vendors=response.json()['vendors']

In [60]:
df = pd.json_normalize(vendors)

In [61]:
df.columns

Index(['alias', 'allowPreorder', 'categoryId', 'cuisines', 'labels', 'logo',
       'name', 'openDays', 'openFrom', 'openTo', 'payments', 'products',
       'chain.alias', 'chain.id.primary', 'cover.1200', 'cover.1600',
       'cover.2048', 'cover.480', 'cover.720', 'delivery.available',
       'delivery.minOrderPrice.value', 'delivery.price.value',
       'delivery.services', 'delivery.time', 'id.primary',
       'reviews.ratingScore', 'reviews.reviewCount', 'reviews.score',
       'reviews.scoreCount'],
      dtype='object')

In [62]:
df.drop(columns=['alias', 'logo', 'openDays', 'openFrom', 'openTo', 'chain.alias', 'chain.id.primary'], inplace=True)
df.drop(columns=['products', 'cover.1200', 'cover.1600', 'cover.2048', 'cover.480', 'cover.720', 'id.primary'], inplace=True)
df.drop(columns=['reviews.ratingScore', 'reviews.reviewCount', 'reviews.score', 'reviews.scoreCount'], inplace=True)

In [63]:
for idx,row in df[:3].iterrows():
    print(idx, row)

0 allowPreorder                                                               False
categoryId                                                                      1
cuisines                                      [Русская, Итальянская, Суши, Пицца]
labels                          [is_fixed_time_delivery, online_payment, by_po...
name                                                                        Му-Му
payments                        [{'type': 'card_online'}, {'type': 'samsungpay...
delivery.available                                                           True
delivery.minOrderPrice.value                                                    1
delivery.price.value                                                           99
delivery.services                                            [delivery, takeaway]
delivery.time                                                           25-35 мин
Name: 0, dtype: object
1 allowPreorder                                                          

In [64]:
cash_free = round(df[df['delivery.price.value'] == 0].shape[0]/df.shape[0] * 100,0)
print(f"Из всех точек только {cash_free} % доставляют шашлык бесплатно")

Из всех точек только 36.0 % доставляют шашлык бесплатно


In [65]:
z_mean = round(df[df['delivery.price.value'] == 0]['delivery.minOrderPrice.value'].mean(), 2)
print(f"Средняя сумма заказа для бесплатной доставки {z_mean} рублей")

Средняя сумма заказа для бесплатной доставки 1411.32 рублей


In [66]:
df.to_csv(PREPARED_DATASET_PATH, index=False, encoding='cp1251', sep=';')